In [2]:
### Allyn J. Schoeffler, PhD
### Assistant Professor
### Loyola University New Orleans

### This code creates visualizations based on SWiLoDD frequency data.
### It is intended for intermediate Python users.
### It should be run via Jupyter Notebook.
### It will plot the difference between average frequencies 
### for a given set of amino acids, but it requires that
### you have already run SWiLoDD for those amino acids
### and saved the results in separate folders with known paths.
### The default mode is to plot two SWiLoDD runs simultaneously,
### but these two analysis runs must have been conducted
### with the same stride and window sizes. Failing to meet this
### requirement will cause spurious results.

# User input fields are near the top. 
# Please read every line carefully:
# Some lines must be edited. 
## Some edits are optional.
### Some lines should not be edited.  

### Package imports; do not edit:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot
import matplotlib.axes
import matplotlib.colors
import warnings
###

### User inputs ###
### Edit the fields below as indicated ###

# Input the  subset headers and the stride you used in the SWiLoDD analysis.
# Subset headers are the names of the two alignment files you submitted to SWiLoDD.
# The script will plot a delta of component 2 minus component 1.

component1_header = 'file_name_of_subset_1'
component2_header = 'file_name_of_subset_2'

# Input the stride you used for SWiLoDD
# Note that a stride of 1 is the default
# Note that all runs must use the same stride for this analysis to work.
stride = 1

# name the output .csv file where computed data will be saved
output_file_name = "SWiLoDD_deltaplot_output.csv"

# Set user-defined cutoffs for significance
# Defaults are already entered below; change if desired.
# Note that this program will automatically apply a Bonferonni cutoff
# based on the number of windows examined in the alignments;
# do not enter a Bonferonni-corrected P-value below.
# The delta cutoff value specifies the smallest difference between alignment partitions
# that you consider biologically significant.
uncorrected_P_cutoff = 0.01
delta_cutoff = 0.07

# Read in the individual .csv outputs from SWiLoDD by specifying file paths.
# These MUST be un-edited 'window-frequencies.csv' files from SWiLoDD runs.
# Two files is the default, but you can add more if desired.
freqs_1df = pd.read_csv("/Users/Name/SWiCAM_v1.1-main/resultsA10/processed_data/window_frequencies.csv")
freqs_2df = pd.read_csv("/Users/Name/SWiCAM_v1.1-main/resultsFYW10/processed_data/window_frequencies.csv")

## Below is optional placeholder code for additional frequency files from SWiLoDD.
## Only use these lines if you are entering frequencies for more than two SWiLoDD runs.
## freqs_3df = pd.read_csv("/Users/Name/SWiCAM_v1.1-main/resultsA10/processed_data/window_frequencies.csv")
## freqs_4df = pd.read_csv("/Users/Name/SWiCAM_v1.1-main/resultsW10/processed_data/window_frequencies.csv")

# Below, add the names of additional dataframes input from csv files, if needed
# No need to edit this if you input two frequency files above (default)
freq_df_list = [freqs_1df, freqs_2df]

## Below is what the line above would look like if you wanted to input four frequency files.
## freq_df_list = [freqs_1df, freqs_2df, freqs_3df, freqs_4df]

# Below, add the names of the residues under investigation in the input csv files, in input order. 
aa = ['A', 'FYW']

## Below is what this line would look like if you instead wanted to examine A, F, Y, and W separately,
## using four separate frequency files from SWiLoDD.
## aa = ['A', 'F', 'Y', 'W']

### Tol Accessible Color Scheme (defaults)
### For more information, see https://davidmathlogic.com/colorblind/#%23D81B60-%231E88E5-%23FFC107-%23004D40 
### These are the default colors; do not edit any of these
TolMaroon = '#882255'
TolSky = '#88CCEE'
TolRose = '#CC6666'
TolBlue = '#332288'
TolGreen = '#117733'
TolGold = '#DDCC77'
TolTeal = '#44AA99'
TolMagenta = '#AA4499'

### These are the defualt color maps; do not edit any of these
TolMaroonMap = matplotlib.colors.LinearSegmentedColormap.from_list('TolMaroonMap', ['white', TolMaroon])
TolBlueMap = matplotlib.colors.LinearSegmentedColormap.from_list('TolBlueMap', ['white', TolBlue])
TolGreenMap = matplotlib.colors.LinearSegmentedColormap.from_list('TolGreenMap', ['white', TolGreen])
TolSkyMap = matplotlib.colors.LinearSegmentedColormap.from_list('TolSkyMap', ['white', TolSky])
TolRoseMap = matplotlib.colors.LinearSegmentedColormap.from_list('TolRoseMap', ['white', TolRose])
TolTealMap = matplotlib.colors.LinearSegmentedColormap.from_list('TolTealMap', ['white', TolTeal])
TolGoldMap = matplotlib.colors.LinearSegmentedColormap.from_list('TolGoldMap', ['white', TolGold])
TolMagentaMap = matplotlib.colors.LinearSegmentedColormap.from_list('TolMagentaMap', ['white', TolMagenta])

# If desired, edit the colors below for your output delta plots
# These colors will correspond to the amino acid groups you are investigating,
# in the order in which you input those amino acid groups.
# Note that typically, the colors and color maps should match, 
# so input colors and their corresponding color maps in the same order.
# You may use named matplotlib colors or hexcodes as well as the default colors defined above. 
# The number of colors you list ***MUST*** match or exceed the number of frequency files you upload!
# (that is, the number of amino acid groups under investigation)

colors = [TolMagenta, TolBlue]
colormaps = [TolMagentaMap, TolBlueMap]

# Edit the colors below as desired for the output histograms
# these colors correspond to your two subsets (aka components).
# user_color1 will map to your component (subset) 1
# user_color2 will map to you component (subset) 2
user_color1 = TolSky
user_color2 = TolRose

## Below are other Tol-scheme colors and colormaps you may wish to use.
## colors = [TolMagenta, TolBlue, TolGreen, TolGold, TolSky, TolRose, TolMaroon, TolTeal]
## colormaps = [TolMagentaMap, TolBlueMap, TolGreenMap, TolGoldMap, TolSkyMap, TolRoseMap, TolMaroonMap, TolTealMap]

#### WARNING!!!! #####
### STOP HERE! ###
### Do not edit the code below unless you are an expert Python user ####

# Code below to force python to treat a RuntimeWarning as an exceptable error.
# This will set any dubious P values to zero and cut down on "false positives" (spuriously tiny P-values)
warnings.simplefilter("error", category=Warning, lineno=0, append=False)

# x axis tick reduction factor
reductX = 25

# This function creates histograms of all wondow sites for which 
# the delta is greater than the user cutoff 
# and the P-value is less than the user cutoff (with the Bonferroni correction applied)
# It will produce histograms for all available csv files input above under User Inputs
# This function also computes deltas 

def delta_sum_data(freq_df_list, aa, subset1_header, subset2_header, uncorrected_P_cutoff, user_cutoff, color1, color2):
    delta_summary_df = pd.DataFrame()
    aa_counter = 0

    for freqs_df in freq_df_list:
        # break into subsets
        subset1_df = freqs_df.loc[freqs_df['subset'] == subset1_header]
        subset2_df = freqs_df.loc[freqs_df['subset'] == subset2_header]
        
        # drop non-numeric columns
        subset1_df_numeric = subset1_df.drop(columns=['subset', 'header'])
        subset2_df_numeric = subset2_df.drop(columns=['subset', 'header'])

        # list and count the windows and compute the Bonferroni cutoff
        windows = list(subset1_df_numeric)
        bonf_cutoff = uncorrected_P_cutoff/len(windows)
        print(bonf_cutoff)
        print(len(windows))
        
        # instantiate lists of statistics for each window
        avg1s = []
        std1s = []
        SEM1s = []
        avg2s = []
        std2s = []
        SEM2s = []
        Ps = []
        ts = []
        color_code_list = []

        #calculate statistics for each window
        
        for window in windows:
            subset1_i_freqlist = subset1_df_numeric[window]
            subset2_i_freqlist = subset2_df_numeric[window]
            avg1s.append(np.mean(subset1_i_freqlist))
            std1s.append(np.std(subset1_i_freqlist))
            SEM1s.append(scipy.stats.sem(subset1_i_freqlist))
            avg2s.append(np.mean(subset2_i_freqlist))
            std2s.append(np.std(subset2_i_freqlist))
            SEM2s.append(scipy.stats.sem(subset2_i_freqlist))
            
            try:
                T, P = scipy.stats.ttest_ind(subset2_i_freqlist, subset1_i_freqlist, equal_var = False)
                Ps.append(P)
                ts.append(T)
            except RuntimeWarning:
                Ps.append(1)
                ts.append(0)
            if P <= bonf_cutoff:
                color_code_list.append(1)
            else:
                color_code_list.append(0)
            
        
        stats_df = pd.DataFrame()
        stats_df.insert(0, '%s avg' % subset1_header, avg1s)
        stats_df.insert(1, '%s std' % subset1_header, std1s)
        stats_df.insert(2, '%s sem' % subset1_header, SEM1s)
        stats_df.insert(3, '%s avg' % subset2_header, avg2s)
        stats_df.insert(4, '%s std' % subset2_header, std2s)
        stats_df.insert(5, '%s sem' % subset2_header, SEM2s)
        stats_df.insert(6, 'delta %s - %s' % (subset2_header, subset1_header), stats_df['%s avg' % subset2_header] - stats_df['%s avg' % subset1_header])
        stats_df.insert(7, 'P values', Ps)
        stats_df.insert(8, 't values', ts)
        
        stats_df.insert(9, 'neg log P', -1*np.log10(stats_df['P values']))
        stats_df.insert(10, 'color_code', color_code_list)

        #calculate the propogated stdev
        series_of_stdevs = np.sqrt((stats_df['%s std' % subset1_header])**2 + (stats_df['%s std' % subset2_header])**2)
        print(series_of_stdevs)
        
        series_of_deltas = stats_df['delta %s - %s' % (subset2_header, subset1_header)]
        series_of_Ps = stats_df['P values']
        series_of_nlPs = stats_df['neg log P']
        series_of_ts = stats_df['t values']
        series_of_color_codes = stats_df['color_code']

        delta_summary_df.insert(0, '%s delta' % aa[aa_counter], series_of_deltas)
        delta_summary_df.insert(0, '%s P values' % aa[aa_counter], series_of_Ps)
        delta_summary_df.insert(0, '%s neglogP' % aa[aa_counter], series_of_nlPs)
        delta_summary_df.insert(0, '%s t value' % aa[aa_counter], series_of_ts)
        delta_summary_df.insert(0, '%s color code' % aa[aa_counter], series_of_color_codes)
        delta_summary_df.insert(0, 'delta stdev %s' % aa[aa_counter], series_of_stdevs)
                
        # plot histograms if the Bonferroni corrected cutoff is acheived
        # turn dataframe index into alignment position usign the user-input stride
        aln_pos_list = [1]
        for n in range(1, len(list(delta_summary_df.index.values))):
            aln_pos_list.append(aln_pos_list[n-1] + stride)
            
        matplotlib.pyplot.rcParams["figure.figsize"] = (20,15)
        window_psn = 1
        for cutoff in list(delta_summary_df['%s color code' % aa[aa_counter]]):
            if cutoff == 1 and abs(list(delta_summary_df['%s delta' % aa[aa_counter]])[window_psn-1]) >= user_cutoff:
                column_head = str(window_psn)
                matplotlib.pyplot.style.use('./hist.mplstyle')
                print(column_head)
                print(aln_pos_list[window_psn-1])
                print(list(delta_summary_df['%s delta' % aa[aa_counter]])[window_psn-1])
                matplotlib.pyplot.hist([list(subset1_df_numeric[column_head]), list(subset2_df_numeric[column_head])], color = [color1, color2], alpha = 0.9, density = True, rwidth = 10)
                matplotlib.pyplot.title('%s aln position %d, P = %.2E' % (aa[aa_counter], aln_pos_list[window_psn-1], list(delta_summary_df['%s P values' % aa[aa_counter]])[window_psn-1]))
                matplotlib.pyplot.ylabel('normalized occurrence')
                matplotlib.pyplot.xlabel("frequency of %s in window" % aa[aa_counter])
                matplotlib.pyplot.savefig("histograms/%s_%s_%s_%d.png" % (subset1_header, subset2_header, aa[aa_counter], aln_pos_list[window_psn-1]))
                matplotlib.pyplot.show()
            window_psn = window_psn + 1
    

        aa_counter = aa_counter + 1
        
    return delta_summary_df

data_summary = delta_sum_data(freq_df_list, aa, component1_header, component2_header, uncorrected_P_cutoff, delta_cutoff, user_color1, user_color2)

#math to turn index into alignment position
aln_pos_list = [1]
for n in range(1, len(list(data_summary.index.values))):
    aln_pos_list.append(aln_pos_list[n-1] + stride)

# The code below uses the data created in the function to create plots of delat values
# for individual csv inputs and for combinations. 

#set figure size & style
matplotlib.pyplot.style.use('./delta.mplstyle')
matplotlib.pyplot.rcParams["figure.figsize"] = (60,21)



##plot individual delta plots
i = 0
for i in range(0, len(aa)):
    #plot points
    matplotlib.pyplot.plot(aln_pos_list, data_summary['%s delta' % aa[i]], alpha = 0.25, color = colors[i], markeredgecolor = colors[i], markerfacecolor = colors[i], marker = "o")
    # color by P-Value cutoff
    matplotlib.pyplot.scatter(aln_pos_list, data_summary['%s delta' % aa[i]], c = data_summary['%s color code' % aa[i]], cmap = colormaps[i])
    matplotlib.pyplot.fill_between(aln_pos_list, data_summary['%s delta' % aa[i]] + data_summary['delta stdev %s' % aa[i]], data_summary['%s delta' % aa[i]] - data_summary['delta stdev %s' % aa[i]], color = colors[i], alpha = 0.15, edgecolor = 'none')
    matplotlib.pyplot.xlim(1, aln_pos_list[-1])
    matplotlib.pyplot.savefig('%s_cut_delta_%s_%s.png' % (aa[i], component2_header, component1_header))
    matplotlib.pyplot.show()
    i = i +1
    
# Plot the combined delta plots
i = 0
for n in aa:
    #plot points
    matplotlib.pyplot.plot(aln_pos_list, data_summary['%s delta' % n], alpha = 0.25, color = colors[i], markeredgecolor = colors[i], markerfacecolor = colors[i], marker = "o")
    # color by P-Value cutoff
    matplotlib.pyplot.scatter(aln_pos_list, data_summary['%s delta' % n], c = data_summary['%s color code' % n], cmap = colormaps[i])
    matplotlib.pyplot.fill_between(aln_pos_list, data_summary['%s delta' % n] + data_summary['delta stdev %s' % n], data_summary['%s delta' % n] - data_summary['delta stdev %s' % n], color = colors[i], alpha = 0.15, edgecolor = 'none')
    i = i+1
matplotlib.pyplot.xlim(1, aln_pos_list[-1])
matplotlib.pyplot.savefig('charged_delta_%s_%s.png' % (component2_header, component1_header))
matplotlib.pyplot.show()


# This line of code saves a csv file of the data used to create the delta plots
data_summary.to_csv(output_file_name)
        

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Name/SWiCAM_v1.1-main/resultsA10/processed_data/window_frequencies.csv'